# Evaluation

* 对不同model计算MSE,MAE,RMSE, 对比模型效果
* 选一个用户，对比已评分商品的实际评分和不同模型下的预测得分
* 选一个用户，展示已评分商品与不同模型下的推荐商品

In [1]:
import pandas as pd
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# 设置中文字体（以 SimHei 为例）
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False  # 显示负号

import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, Embedding, Flatten, RNN, Multiply, Dot,Reshape
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout
from sklearn.metrics import mean_squared_error,mean_absolute_error
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.models import Model
from keras.layers import Input, Embedding, Dot, Flatten, Add
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
import tensorflow.keras.backend as K
from keras.callbacks import EarlyStopping

### 对不同model计算MSE,MAE,RMSE, 对比模型效果

In [3]:
from keras.models import load_model

# 加载模型
model_alltogether_mlp = load_model('alltogether_mlp.h5')
model_user_product_mlp = load_model('user_product_mlp.h5')
model_user_product_rnn_mlp = load_model('user_product_rnn_mlp.h5')
model_alltogether_mlp_BERT = load_model('alltogether_mlp_BERT.h5')
model_user_product_mlp_BERT = load_model('user_product_mlp_BERT.h5')

models = [model_alltogether_mlp, model_user_product_mlp, model_user_product_rnn_mlp,model_alltogether_mlp_BERT,model_user_product_mlp_BERT]
model_names = ['all_together_mlp', 'user_product_mlp', 'user_product_rnn_mlp','model_alltogether_mlp_BERT','model_user_product_mlp_BERT']

# 加载 Tokenizer
with open('tokenizer_title.pkl', 'rb') as f:
    loaded_tokenizer_title = pickle.load(f)
    
with open('tokenizer_summary.pkl', 'rb') as f:
    loaded_tokenizer_summary = pickle.load(f)

In [4]:
# 读取测试集
df_test = pd.read_parquet('data_test.parquet')

# 拼接summary
long_summary = pd.read_parquet('long_summary.parquet')
df_test = pd.merge(df_test,long_summary,on="productId_encode",how="left")

# 拼接BERT训练的title和summary
title_bert = pd.read_parquet('title_embeded_df.parquet')
title_bert = title_bert.rename(columns=lambda x: f'title_{x}' if x != 'productId_encode' else x)
summary_bert = pd.read_parquet('summary_embeded_df.parquet')
summary_bert = summary_bert.rename(columns=lambda x: f'summary_{x}' if x != 'productId_encode' else x)
df_test = pd.merge(df_test,title_bert,on="productId_encode",how="left")
df_test = pd.merge(df_test,summary_bert,on="productId_encode",how="left")

# 变量名
column_names = df_test.columns.tolist()
# user variable
user_columns =  ['userId_encode']+ [elem for elem in column_names if elem.startswith('his_')]
# product variable (without text)
product_nontext_columns = ['productId_encode','train_mean_score','helpfulness_helpful','helpfulness_total']+[elem for elem in column_names if elem.startswith('categories_')]+[elem for elem in column_names if elem.startswith('brand_')]
product_nontext_columns_BERT = ['productId_encode','train_mean_score','helpfulness_helpful','helpfulness_total']+[elem for elem in column_names if elem.startswith('categories_')]+[elem for elem in column_names if elem.startswith('brand_')]+ [f'title_{i}' for i in range(768)]+[f'summary_{i}' for i in range(768)]
# target
target_column = 'score'

# 输入变量
max_length_title = 50
X_product_title_test = pad_sequences(loaded_tokenizer_title.texts_to_sequences(df_test['title']), maxlen=max_length_title)
max_length_summary = 100
X_product_summary_test = pad_sequences(loaded_tokenizer_summary.texts_to_sequences(df_test['long_summary']), maxlen=max_length_summary)

X_user_test = df_test[user_columns].values
X_product_nontext_test = df_test[product_nontext_columns].values
X_product_nontext_test_BERT = df_test[product_nontext_columns_BERT].values

# 进行预测
predictions_alltogether_mlp = model_alltogether_mlp.predict([X_user_test,X_product_nontext_test])
predictions_user_product_mlp = model_user_product_mlp.predict([X_user_test,X_product_nontext_test])
predictions_user_product_rnn_mlp = model_user_product_rnn_mlp.predict([X_user_test,X_product_nontext_test,X_product_title_test,X_product_summary_test])
predictions_alltogether_mlp_BERT = model_alltogether_mlp_BERT.predict([X_user_test,X_product_nontext_test_BERT])
predictions_user_product_mlp_BERT = model_user_product_mlp_BERT.predict([X_user_test,X_product_nontext_test_BERT])

predictions = [predictions_alltogether_mlp,predictions_user_product_mlp,predictions_user_product_rnn_mlp,predictions_alltogether_mlp_BERT,predictions_user_product_mlp_BERT]

637/637 [==============================] - 1s 2ms/step


In [5]:
def evaluation(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse,mae,rmse

# Store results
results = []

# Evaluate each model's predictions
models_predictions = [
    ('All Together MLP', predictions_alltogether_mlp),
    ('User-Product MLP', predictions_user_product_mlp),
    ('User-Product RNN MLP', predictions_user_product_rnn_mlp),
    ('User-Product MLP BERT', predictions_user_product_mlp_BERT)
]

results.append({'Model': 'BiasSVD', 'MSE': 1.9072885680078813, 'MAE': 0.961590217044084, 'RMSE': 1.3810461860516763})

for model_name, y_pred in models_predictions:
    mse, mae, rmse = evaluation(df_test[target_column], y_pred)
    results.append({'Model': model_name, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

# Create DataFrame and print it
results_df = pd.DataFrame(results)
display(results_df)

,Model,MSE,MAE,RMSE
0,BiasSVD,1.907289,0.961590,1.381046
1,All Together MLP,1.596291,1.104165,1.263444
2,User-Product MLP,1.417999,0.978643,1.190798
3,User-Product RNN MLP,1.281766,0.869168,1.132151
4,User-Product MLP BERT,1.459245,1.014366,1.207992


### 选一个用户，对比已评分商品的实际评分和不同模型下的预测得分

In [11]:
# 获取唯一的 productId_encode，计算有多少product
unique_product_ids = df_test['productId_encode'].drop_duplicates(inplace=False)
n_products = unique_product_ids.shape[0]
print('n_products',n_products)
# 根据 unique_product_ids 筛选 df_test
productsId_unique_df = df_test[df_test['productId_encode'].isin(unique_product_ids)].drop_duplicates(subset='productId_encode').reset_index(drop=True)

n_products 8515


In [7]:
def get_user_true_and_predict_score(sample_userid, models, model_names):
    # 获取用户特征
    user_features = df_test[df_test['userId_encode'] == sample_userid][user_columns].values[0]
    user_features_tiled = np.tile(user_features, (n_products, 1))

    # Store predictions
    predictions = {}

    # 利用模型对该用户对所有商品的预测打分
    for model, model_name in zip(models, model_names):
        if model_name == 'user_product_rnn_mlp':
            predictions[model_name] = model.predict([
                user_features_tiled,
                productsId_unique_df[product_nontext_columns].values,
                pad_sequences(loaded_tokenizer_title.texts_to_sequences(productsId_unique_df['title']), maxlen=max_length_title),     
                pad_sequences(loaded_tokenizer_summary.texts_to_sequences(productsId_unique_df['long_summary']), maxlen=max_length_summary)
            ])
        elif model_name =='model_alltogether_mlp_BERT' or model_name == 'model_user_product_mlp_BERT':
            predictions[model_name] = model.predict([
                user_features_tiled,
                productsId_unique_df[product_nontext_columns_BERT].values
            ])
        else:
            predictions[model_name] = model.predict([
                user_features_tiled,
                productsId_unique_df[product_nontext_columns].values
            ])
            
    # Create DataFrame for predictions
    prediction_dfs = [pd.DataFrame(predictions[name], columns=[f'predicted_score_{name}']) for name in model_names]
    result = pd.concat([productsId_unique_df] + prediction_dfs, axis=1)
    result = result[['productId_encode'] + [f'predicted_score_{name}' for name in model_names]]

    # 获取用户已评分的产品
    user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode', 'score']].reset_index(drop=True)

    # 使用 merge 筛选出已经评分的产品的预测分数
    rated_products_with_predictions = user_rated_products.merge(
        result,
        on='productId_encode',
        how='inner'
    )

    # 根据 'score' 对 rated_products_with_predictions 进行排序
    rated_products_with_predictions_sorted = rated_products_with_predictions.sort_values(by='score', ascending=False).reset_index(drop=True)

    # 显示筛选结果
    display(rated_products_with_predictions_sorted)

In [8]:
# 计算每个用户的评分次数
user_rating_counts = df_test.groupby('userId_encode').size()
# 筛选出评分次数多于 3 次的用户
users_with_many_ratings = user_rating_counts[user_rating_counts > 3].index

In [9]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
get_user_true_and_predict_score(sample_userid, models, model_names)

sample_userid 133975
267/267 [==============================] - 0s 2ms/step


,productId_encode,score,predicted_score_all_together_mlp,predicted_score_user_product_mlp,predicted_score_user_product_rnn_mlp,predicted_score_model_alltogether_mlp_BERT,predicted_score_model_user_product_mlp_BERT
0,2522,5.0,3.907541,4.168075,4.128696,4.137772,4.122420
1,13647,5.0,3.776849,4.038621,4.493172,4.080497,3.966675
2,26914,4.0,3.742747,3.948262,4.158074,3.952252,3.894739
3,17169,4.0,3.754728,3.995464,4.499241,4.062783,3.947793
4,25417,4.0,3.737122,3.948269,4.484288,3.970607,3.899946


In [10]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
get_user_true_and_predict_score(sample_userid, models, model_names)

sample_userid 125814
267/267 [==============================] - 0s 2ms/step


,productId_encode,score,predicted_score_all_together_mlp,predicted_score_user_product_mlp,predicted_score_user_product_rnn_mlp,predicted_score_model_alltogether_mlp_BERT,predicted_score_model_user_product_mlp_BERT
0,2832,5.0,3.866900,4.167475,4.529414,4.163741,4.177632
1,36802,5.0,3.754286,3.829693,4.532549,3.854924,3.860695
2,2933,5.0,3.865801,4.166622,4.536425,4.162351,4.157618
3,2833,5.0,3.866897,4.167402,4.536978,4.164201,4.180215


In [11]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
get_user_true_and_predict_score(sample_userid, models, model_names)

sample_userid 123752
267/267 [==============================] - 0s 2ms/step


,productId_encode,score,predicted_score_all_together_mlp,predicted_score_user_product_mlp,predicted_score_user_product_rnn_mlp,predicted_score_model_alltogether_mlp_BERT,predicted_score_model_user_product_mlp_BERT
0,14870,5.0,3.741900,4.023635,4.525764,4.063698,3.961977
1,30754,5.0,3.738146,3.897946,3.725980,3.891493,3.876955
2,30672,5.0,3.737599,3.902051,4.524405,3.904359,3.891227
3,14859,5.0,3.741971,4.023782,4.510567,4.063944,3.961747


In [12]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
get_user_true_and_predict_score(sample_userid, models, model_names)

sample_userid 125219
267/267 [==============================] - 0s 2ms/step


,productId_encode,score,predicted_score_all_together_mlp,predicted_score_user_product_mlp,predicted_score_user_product_rnn_mlp,predicted_score_model_alltogether_mlp_BERT,predicted_score_model_user_product_mlp_BERT
0,14115,5.0,3.750724,4.034255,3.655637,4.055370,3.935145
1,21027,5.0,3.724207,3.961730,4.520035,4.020805,3.923033
2,7041,5.0,3.812521,4.130783,4.467666,4.128724,3.980785
3,8213,5.0,3.801814,4.128120,4.478900,4.145808,4.155247
4,5140,3.0,3.836454,4.150455,4.517040,4.148531,4.069329


In [13]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
get_user_true_and_predict_score(sample_userid, models, model_names)

sample_userid 88081
267/267 [==============================] - 0s 2ms/step


,productId_encode,score,predicted_score_all_together_mlp,predicted_score_user_product_mlp,predicted_score_user_product_rnn_mlp,predicted_score_model_alltogether_mlp_BERT,predicted_score_model_user_product_mlp_BERT
0,3150,5.0,3.922410,4.172466,4.540308,4.090387,4.146919
1,3142,5.0,3.922469,4.172526,4.543626,4.090424,4.147027
2,3145,5.0,3.922447,4.172503,4.540308,4.090410,4.146987
3,3146,5.0,3.922440,4.172496,4.543626,4.090405,4.146974


### 选一个用户，展示已评分商品与不同模型下的推荐商品

In [24]:
def recommend(sample_userid,model,model_name):
    # 获取用户特征
    user_features = df_test[df_test['userId_encode'] == sample_userid][user_columns].values[0]
    user_features_tiled = np.tile(user_features, (n_products, 1))

    if model_name == 'user_product_rnn_mlp':
        predictions = model.predict([
            user_features_tiled,
            productsId_unique_df[product_nontext_columns].values,
            pad_sequences(loaded_tokenizer_title.texts_to_sequences(productsId_unique_df['title']), maxlen=max_length_title),     
            pad_sequences(loaded_tokenizer_summary.texts_to_sequences(productsId_unique_df['long_summary']), maxlen=max_length_summary)
        ])
    elif model_name =='model_alltogether_mlp_BERT' or model_name == 'model_user_product_mlp_BERT':
        predictions = model.predict([
            user_features_tiled,
            productsId_unique_df[product_nontext_columns_BERT].values
        ])
    else:
        predictions = model.predict([
            user_features_tiled,
            productsId_unique_df[product_nontext_columns].values
        ])
            
    # 创建带预测分数的 DataFrame，并排序
    prediction_column_name = 'pred_score'
    result = pd.concat([productsId_unique_df]+[pd.DataFrame(predictions, columns=[prediction_column_name])], axis=1)
    result_sorted = result.sort_values(by=prediction_column_name, ascending=False)

    # 只显示 product_id 和 predictions 列,前十个推荐的商品
    display(result_sorted[['productId_encode','title', prediction_column_name]].head(10))

In [15]:
# 设置最大列宽
pd.set_option('display.max_colwidth', None)  # None表示不限制宽度

In [15]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
# 该用户历史评分的产品
user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode','title', 'score']].reset_index(drop=True)
display(user_rated_products)
# 用各个模型给他推荐产品
for model, model_name in zip(models, model_names):
    recommend(sample_userid,model,model_name)

sample_userid 119139


,productId_encode,title,score
0,1063,"14k White Gold 5mm Half Round Plain Wedding Band, Size 8",5.0
1,1070,"14k White Gold 5mm Half Round Plain Wedding Band, Size 12",5.0
2,1061,"14k White Gold 5mm Half Round Plain Wedding Band, Size 6.5",5.0
3,1062,"14k White Gold 5mm Half Round Plain Wedding Band, Size 7",5.0
4,1058,"14k White Gold 5mm Half Round Plain Wedding Band, Size 4.5",5.0
5,1068,"14k White Gold 5mm Half Round Plain Wedding Band, Size 10.5",5.0


267/267 [==============================] - 0s 1ms/step


,productId_encode,title,predictions_all_together_mlp
1972,69,Jabra EarGels - Clear,3.862248
3200,18,Music Maker Lap Harp,3.861892
5931,6,Mechanical and Quartz Watch Repair,3.861664
484,23,Plantronics Mobile Convertible Headset M175C,3.861604
3574,21,Sony ECMMS907 Digital Recording Microphone,3.861571
6417,20,PLANTRONICS M-170 Convertible Headset with Noise-Canceling Boom Microphones (PLANTRONICS M170),3.861515
4222,15,"Remo Kid's Percussion, Bongos, 5/6 Diameters, Rain Forest Fabric",3.861506
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,3.861506
1140,12,Citizen Men's Eco-Drive Stiletto watch #AR1004-51E,3.861495
7453,16,Harmonica,3.861467


267/267 [==============================] - 0s 1ms/step


...

,productId_encode,title,predictions_user_product_rnn_mlp
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.564329
5281,36060,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 6, #10825",4.560808
7387,36062,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 7, #10825",4.560036
6021,6293,Seymour Duncan STHR1B Hot Rails Lead Pickup for Telecaster,4.556239
3448,35921,Loving Family&reg Large Sterling Silver Pendant with Extender - Parents and 3 Children,4.555840
6174,33043,"Harley-Davidson Bulova Mens Watch, Antique Medallion Skull Dial, 76A11",4.554208
5339,23964,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10 1/2, #11873",4.553726
5491,17245,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10, #11873",4.553697
3607,17248,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 11, #11873",4.553627
6813,33726,"Sterling Silver Round Clear Cubic Zirconia Initial ""M"" Charm 1 inch Long Pendant",4.553259


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_model_alltogether_mlp_BERT
3116,311,Shure M97xE High-Performance Magnetic Phono Cartridge,4.212601
1972,69,Jabra EarGels - Clear,4.212588
627,673,Cellular Innovations A-BOOSTER Universal Cell Phone Antenna Booster,4.207930
2791,610,Yamaha FG JR1 3/4 Size Acoustic Guitar with Gig Bag,4.205259
130,291,Audio-Technica ATR-35S Lavalier Microphone,4.204763
621,1204,Duragold 14k Yellow Gold or White Gold or Two-Tone Satin and Polished Crossover Hoop Earrings,4.204731
3261,1191,Duragold 14k Yellow Gold Half Round Hoop Earrings,4.201962
4602,1193,"Duragold 14k Gold Hoop Earrings (0.59"" Diameter)",4.201871
3153,283,Audio Technica ATR288W VHF Battery-Powered TwinMic Microphone System,4.200544
3200,18,Music Maker Lap Harp,4.199145


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_model_user_product_mlp_BERT
7465,178,"Fender 18 Ft. Vintage Voltage Guitar Cable, Black/Silver",4.207655
4883,442,Silver Chinese Ring,4.188780
8435,445,Silver Chinese Ring,4.188766
7376,446,Silver Chinese Ring,4.188760
3756,447,Silver Chinese Ring,4.188756
6097,514,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.188594
4013,517,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.188581
930,546,Oval-Shaped Genuine Coral 18k Yellow Gold Over Sterling Silver Cluster Cocktail Ring,4.188318
2197,505,PalmBeach Jewelry 5.81 TCW Oval Cut Aurora Borealis Cubic Zirconia Sterling Silver Cocktail Ring,4.188179
3830,443,Silver Chinese Ring,4.188096


In [25]:
sample_userid = 125326

# 该用户历史评分的产品
user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode','title', 'score']].reset_index(drop=True)
display(user_rated_products)
# 用各个模型给他推荐产品
for model, model_name in zip(models, model_names):
    recommend(sample_userid,model,model_name)

,productId_encode,title,score
0,7144,Yamaha YRA-314B Alto 3 Piece Recorder - Simulated Ebony,5.0
1,16862,Korg TM-40 Large Display Digital Tuner and Metronome,4.0
2,7177,"Yamaha YAC-1071P Flute Key Plugs, 10Pk",5.0
3,10104,Yamaha YRB-302B Bass Recorder - Baroque 4 Piece Model,5.0
4,6735,Bo-Pep Flute Finger Saddle,3.0
5,6535,On-Stage Stands Clarinet or Flute Stand,5.0
6,6736,Bo-Pep Flute Thumb Guide,4.0
7,21110,Hercules DS502B Flute/Clarinetinet PEG,5.0
8,4590,Auralex D36CHA/PUR Designer Series Roominator Kit: 18-DST 1'x1'x2 DST112 Panels- Charcoal; 18-1'x1'x2 DST114 Panel- Purple; 2-Tubes of Tubetak Pro,4.0
9,11495,"Yamaha YRS-312B Soprano Recorder, Key of C",5.0


267/267 [==============================] - 0s 1ms/step


,productId_encode,title,pred_score
1972,69,Jabra EarGels - Clear,3.895934
3200,18,Music Maker Lap Harp,3.895578
5931,6,Mechanical and Quartz Watch Repair,3.895350
484,23,Plantronics Mobile Convertible Headset M175C,3.895289
3574,21,Sony ECMMS907 Digital Recording Microphone,3.895257
6417,20,PLANTRONICS M-170 Convertible Headset with Noise-Canceling Boom Microphones (PLANTRONICS M170),3.895201
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,3.895192
4222,15,"Remo Kid's Percussion, Bongos, 5/6 Diameters, Rain Forest Fabric",3.895191
1140,12,Citizen Men's Eco-Drive Stiletto watch #AR1004-51E,3.895180
7453,16,Harmonica,3.895153


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,pred_score
4888,43,Technics SL1200M3D Turntable,4.185781
3096,46,First Act Player Series Tambourine,4.185764
5990,49,Techno eJay,4.185728
703,61,Nokia LCH-12 Car Charger for Nokia Phones and Bluetooth Headsets,4.185678
1140,12,Citizen Men's Eco-Drive Stiletto watch #AR1004-51E,4.185635
6816,81,First Act Acoustic Guitar Mg311,4.185622
2615,73,Monster Standard 100 Instrument Cable 21 ft. - straight 1/4 plugs S100-I-21,4.185598
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.185585
6683,109,Yamaha PSR-170 61-Key Portable Electronic Keyboard,4.185531
3228,94,Jabra EarBud for phones with Universal 2.5mm Jack,4.185492


...

,productId_encode,title,pred_score
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.564196
5281,36060,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 6, #10825",4.560711
7387,36062,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 7, #10825",4.559938
6021,6293,Seymour Duncan STHR1B Hot Rails Lead Pickup for Telecaster,4.556145
3448,35921,Loving Family&reg Large Sterling Silver Pendant with Extender - Parents and 3 Children,4.555743
6174,33043,"Harley-Davidson Bulova Mens Watch, Antique Medallion Skull Dial, 76A11",4.554111
5339,23964,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10 1/2, #11873",4.553631
5491,17245,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10, #11873",4.553603
3607,17248,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 11, #11873",4.553533
6813,33726,"Sterling Silver Round Clear Cubic Zirconia Initial ""M"" Charm 1 inch Long Pendant",4.553161


267/267 [==============================] - 1s 2ms/step


,productId_encode,title,pred_score
3116,311,Shure M97xE High-Performance Magnetic Phono Cartridge,4.191884
1972,69,Jabra EarGels - Clear,4.191354
627,673,Cellular Innovations A-BOOSTER Universal Cell Phone Antenna Booster,4.186074
2791,610,Yamaha FG JR1 3/4 Size Acoustic Guitar with Gig Bag,4.183425
3200,18,Music Maker Lap Harp,4.183128
130,291,Audio-Technica ATR-35S Lavalier Microphone,4.182943
621,1204,Duragold 14k Yellow Gold or White Gold or Two-Tone Satin and Polished Crossover Hoop Earrings,4.182900
3783,403,PalmBeach Jewelry 18k Yellow Gold-Plated Two-Tone Filigree Butterfly Ring,4.181624
4024,55,Yamaha YRS24B Soprano Recorder,4.181333
7443,464,4.90 TCW Oval Cut Genuine Smoky Quartz 14k Yellow Gold Over Sterling Silver 3-Stone Ring,4.180957


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,pred_score
7465,178,"Fender 18 Ft. Vintage Voltage Guitar Cable, Black/Silver",4.219281
4883,442,Silver Chinese Ring,4.194049
8435,445,Silver Chinese Ring,4.194032
7376,446,Silver Chinese Ring,4.194026
3756,447,Silver Chinese Ring,4.194021
6097,514,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.193820
4013,517,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.193812
930,546,Oval-Shaped Genuine Coral 18k Yellow Gold Over Sterling Silver Cluster Cocktail Ring,4.193504
2197,505,PalmBeach Jewelry 5.81 TCW Oval Cut Aurora Borealis Cubic Zirconia Sterling Silver Cocktail Ring,4.193369
3830,443,Silver Chinese Ring,4.193289


In [17]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
# 该用户历史评分的产品
user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode','title', 'score']].reset_index(drop=True)
display(user_rated_products)
# 用各个模型给他推荐产品
for model, model_name in zip(models, model_names):
    recommend(sample_userid,model,model_name)

sample_userid 125326


,productId_encode,title,score
0,7144,Yamaha YRA-314B Alto 3 Piece Recorder - Simulated Ebony,5.0
1,16862,Korg TM-40 Large Display Digital Tuner and Metronome,4.0
2,7177,"Yamaha YAC-1071P Flute Key Plugs, 10Pk",5.0
3,10104,Yamaha YRB-302B Bass Recorder - Baroque 4 Piece Model,5.0
4,6735,Bo-Pep Flute Finger Saddle,3.0
5,6535,On-Stage Stands Clarinet or Flute Stand,5.0
6,6736,Bo-Pep Flute Thumb Guide,4.0
7,21110,Hercules DS502B Flute/Clarinetinet PEG,5.0
8,4590,Auralex D36CHA/PUR Designer Series Roominator Kit: 18-DST 1'x1'x2 DST112 Panels- Charcoal; 18-1'x1'x2 DST114 Panel- Purple; 2-Tubes of Tubetak Pro,4.0
9,11495,"Yamaha YRS-312B Soprano Recorder, Key of C",5.0


267/267 [==============================] - 0s 1ms/step


,productId_encode,title,predictions_all_together_mlp
1972,69,Jabra EarGels - Clear,3.895934
3200,18,Music Maker Lap Harp,3.895578
5931,6,Mechanical and Quartz Watch Repair,3.895350
484,23,Plantronics Mobile Convertible Headset M175C,3.895289
3574,21,Sony ECMMS907 Digital Recording Microphone,3.895257
6417,20,PLANTRONICS M-170 Convertible Headset with Noise-Canceling Boom Microphones (PLANTRONICS M170),3.895201
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,3.895192
4222,15,"Remo Kid's Percussion, Bongos, 5/6 Diameters, Rain Forest Fabric",3.895191
1140,12,Citizen Men's Eco-Drive Stiletto watch #AR1004-51E,3.895180
7453,16,Harmonica,3.895153


267/267 [==============================] - 0s 1ms/step


...

,productId_encode,title,predictions_user_product_rnn_mlp
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.564196
5281,36060,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 6, #10825",4.560711
7387,36062,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 7, #10825",4.559938
6021,6293,Seymour Duncan STHR1B Hot Rails Lead Pickup for Telecaster,4.556145
3448,35921,Loving Family&reg Large Sterling Silver Pendant with Extender - Parents and 3 Children,4.555743
6174,33043,"Harley-Davidson Bulova Mens Watch, Antique Medallion Skull Dial, 76A11",4.554111
5339,23964,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10 1/2, #11873",4.553631
5491,17245,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10, #11873",4.553603
3607,17248,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 11, #11873",4.553533
6813,33726,"Sterling Silver Round Clear Cubic Zirconia Initial ""M"" Charm 1 inch Long Pendant",4.553161


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_model_alltogether_mlp_BERT
3116,311,Shure M97xE High-Performance Magnetic Phono Cartridge,4.191884
1972,69,Jabra EarGels - Clear,4.191354
627,673,Cellular Innovations A-BOOSTER Universal Cell Phone Antenna Booster,4.186074
2791,610,Yamaha FG JR1 3/4 Size Acoustic Guitar with Gig Bag,4.183425
3200,18,Music Maker Lap Harp,4.183128
130,291,Audio-Technica ATR-35S Lavalier Microphone,4.182943
621,1204,Duragold 14k Yellow Gold or White Gold or Two-Tone Satin and Polished Crossover Hoop Earrings,4.182900
3783,403,PalmBeach Jewelry 18k Yellow Gold-Plated Two-Tone Filigree Butterfly Ring,4.181624
4024,55,Yamaha YRS24B Soprano Recorder,4.181333
7443,464,4.90 TCW Oval Cut Genuine Smoky Quartz 14k Yellow Gold Over Sterling Silver 3-Stone Ring,4.180957


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_model_user_product_mlp_BERT
7465,178,"Fender 18 Ft. Vintage Voltage Guitar Cable, Black/Silver",4.219281
4883,442,Silver Chinese Ring,4.194049
8435,445,Silver Chinese Ring,4.194032
7376,446,Silver Chinese Ring,4.194026
3756,447,Silver Chinese Ring,4.194021
6097,514,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.193820
4013,517,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.193812
930,546,Oval-Shaped Genuine Coral 18k Yellow Gold Over Sterling Silver Cluster Cocktail Ring,4.193504
2197,505,PalmBeach Jewelry 5.81 TCW Oval Cut Aurora Borealis Cubic Zirconia Sterling Silver Cocktail Ring,4.193369
3830,443,Silver Chinese Ring,4.193289


In [18]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
# 该用户历史评分的产品
user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode','title', 'score']].reset_index(drop=True)
display(user_rated_products)
# 用各个模型给他推荐产品
for model, model_name in zip(models, model_names):
    recommend(sample_userid,model,model_name)

sample_userid 27260


,productId_encode,title,score
0,5139,Elixir Light Nanoweb 80/20 Bronze Acoustic Guitar Strings,5.0
1,5174,Mogami Gold Studio 25 Microphone Cable Quad Conductor 25 feet,4.0
2,5151,"Elixir Strings Acoustic Guitar Strings, 6-String, Extra Light NANOWEB Coating",5.0
3,32677,"Dunlop 272 Traditional Wall Blues Bottle Slide, Medium",5.0


267/267 [==============================] - 0s 1ms/step


,productId_encode,title,predictions_all_together_mlp
6497,25706,Invicta Men's 3441 Pro Diver Collection Watch,3.719826
5218,25691,Invicta Men's 3196 Pro Diver Collection Commemorative Edition Watch,3.719813
910,25735,Invicta Men's 8946 II Collection Camelot 23k Gold Stainless Steel Watch,3.719810
4112,25700,Invicta Men's 9816 Lupah Collection Chronograph Watch,3.719808
663,25752,Invicta Men's 3490 II Collection Orange Canvas Watch,3.719803
3546,25708,Invicta Women's 8944 Pro Diver Collection Gold-Tone Watch,3.719801
4561,25712,Jewelry Box Travel Case with Lock and Key in Red Sale,3.719800
3048,25677,Invicta Men's 3657 II Collection Multi-Function Two-Tone Watch,3.719784
4169,25681,Invicta Men's 8935 Pro Diver Collection Two-Tone Watch,3.719776
281,25760,Invicta Men's 9332 Camelot Citadel Stainless Steel Watch,3.719772


267/267 [==============================] - 0s 2ms/step


...

,productId_encode,title,predictions_user_product_rnn_mlp
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.563215
5281,36060,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 6, #10825",4.558885
7387,36062,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 7, #10825",4.558118
6021,6293,Seymour Duncan STHR1B Hot Rails Lead Pickup for Telecaster,4.554542
3448,35921,Loving Family&reg Large Sterling Silver Pendant with Extender - Parents and 3 Children,4.553952
6174,33043,"Harley-Davidson Bulova Mens Watch, Antique Medallion Skull Dial, 76A11",4.552325
5491,17245,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10, #11873",4.551932
3607,17248,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 11, #11873",4.551864
5339,23964,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10 1/2, #11873",4.551857
6813,33726,"Sterling Silver Round Clear Cubic Zirconia Initial ""M"" Charm 1 inch Long Pendant",4.551379


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_model_alltogether_mlp_BERT
7453,16,Harmonica,4.232603
5931,6,Mechanical and Quartz Watch Repair,4.232585
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.232577
4222,15,"Remo Kid's Percussion, Bongos, 5/6 Diameters, Rain Forest Fabric",4.232521
1140,12,Citizen Men's Eco-Drive Stiletto watch #AR1004-51E,4.232503
3698,19,Hohner Kids Beginner Harmonica,4.232466
3096,46,First Act Player Series Tambourine,4.232408
5445,92,Suunto Yachtsman Wrist Top Computer Watch with Barometer and Compass,4.232347
4888,43,Technics SL1200M3D Turntable,4.232339
7465,178,"Fender 18 Ft. Vintage Voltage Guitar Cable, Black/Silver",4.232317


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_model_user_product_mlp_BERT
4883,442,Silver Chinese Ring,4.200913
8435,445,Silver Chinese Ring,4.200894
7376,446,Silver Chinese Ring,4.200889
3756,447,Silver Chinese Ring,4.200882
6097,514,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.200710
4013,517,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.200677
930,546,Oval-Shaped Genuine Coral 18k Yellow Gold Over Sterling Silver Cluster Cocktail Ring,4.200356
2197,505,PalmBeach Jewelry 5.81 TCW Oval Cut Aurora Borealis Cubic Zirconia Sterling Silver Cocktail Ring,4.200127
3830,443,Silver Chinese Ring,4.199992
2299,444,Silver Chinese Ring,4.199985


In [19]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
# 该用户历史评分的产品
user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode','title', 'score']].reset_index(drop=True)
display(user_rated_products)
# 用各个模型给他推荐产品
for model, model_name in zip(models, model_names):
    recommend(sample_userid,model,model_name)

sample_userid 126570


,productId_encode,title,score
0,23299,Bulova Men's 98C60 Two-Tone Bracelet Watch,4.0
1,23374,Bulova Men's 97C48 Bracelet Watch,4.0
2,25850,Seiko Men's SNXJ90 5 Automatic Two-Tone Stainless Steel Watch,4.0
3,12120,Seiko Men's SGF206 Dress Gold-Tone Watch,3.0
4,24090,Casio Men's G300-3AV G-Shock Ana-Digi Black Street Rider Watch,4.0
5,12098,Seiko Men's SKX173 Automatic Dive Urethane Strap Watch,4.0
6,32166,Anne Klein Women's 10-7605CHRM Swarovski Crystal Silver-Tone Charm Bracelet Watch,4.0
7,21837,Citizen Men's BN0000-04H Eco-Drive Professional Diver Black Rubber Strap Watch,4.0
8,24036,Casio Men's G300-2AV G-Shock Ana-Digi Shock Resistant Street Rider Sports Watch,4.0
9,24343,Bulova Men's 97B13 Black Tortoise Shell Leather Watch,4.0


267/267 [==============================] - 0s 1ms/step


,productId_encode,title,predictions_all_together_mlp
1972,69,Jabra EarGels - Clear,3.902784
3200,18,Music Maker Lap Harp,3.902428
5931,6,Mechanical and Quartz Watch Repair,3.902200
484,23,Plantronics Mobile Convertible Headset M175C,3.902139
3574,21,Sony ECMMS907 Digital Recording Microphone,3.902107
6417,20,PLANTRONICS M-170 Convertible Headset with Noise-Canceling Boom Microphones (PLANTRONICS M170),3.902051
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,3.902042
4222,15,"Remo Kid's Percussion, Bongos, 5/6 Diameters, Rain Forest Fabric",3.902042
1140,12,Citizen Men's Eco-Drive Stiletto watch #AR1004-51E,3.902030
7453,16,Harmonica,3.902002


267/267 [==============================] - 0s 2ms/step


...

,productId_encode,title,predictions_user_product_rnn_mlp
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.564170
5281,36060,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 6, #10825",4.560692
7387,36062,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 7, #10825",4.559920
6021,6293,Seymour Duncan STHR1B Hot Rails Lead Pickup for Telecaster,4.556128
3448,35921,Loving Family&reg Large Sterling Silver Pendant with Extender - Parents and 3 Children,4.555725
6174,33043,"Harley-Davidson Bulova Mens Watch, Antique Medallion Skull Dial, 76A11",4.554092
5339,23964,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10 1/2, #11873",4.553614
5491,17245,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10, #11873",4.553585
3607,17248,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 11, #11873",4.553515
6813,33726,"Sterling Silver Round Clear Cubic Zirconia Initial ""M"" Charm 1 inch Long Pendant",4.553144


267/267 [==============================] - 1s 2ms/step


,productId_encode,title,predictions_model_alltogether_mlp_BERT
3116,311,Shure M97xE High-Performance Magnetic Phono Cartridge,4.187753
1972,69,Jabra EarGels - Clear,4.187157
627,673,Cellular Innovations A-BOOSTER Universal Cell Phone Antenna Booster,4.181876
2791,610,Yamaha FG JR1 3/4 Size Acoustic Guitar with Gig Bag,4.179366
3200,18,Music Maker Lap Harp,4.179069
130,291,Audio-Technica ATR-35S Lavalier Microphone,4.178884
621,1204,Duragold 14k Yellow Gold or White Gold or Two-Tone Satin and Polished Crossover Hoop Earrings,4.178831
3783,403,PalmBeach Jewelry 18k Yellow Gold-Plated Two-Tone Filigree Butterfly Ring,4.177565
4024,55,Yamaha YRS24B Soprano Recorder,4.177273
7443,464,4.90 TCW Oval Cut Genuine Smoky Quartz 14k Yellow Gold Over Sterling Silver 3-Stone Ring,4.176898


267/267 [==============================] - 1s 2ms/step


,productId_encode,title,predictions_model_user_product_mlp_BERT
7465,178,"Fender 18 Ft. Vintage Voltage Guitar Cable, Black/Silver",4.222247
4883,442,Silver Chinese Ring,4.195117
8435,445,Silver Chinese Ring,4.195109
7376,446,Silver Chinese Ring,4.195107
3756,447,Silver Chinese Ring,4.195105
6097,514,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.195026
4013,517,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.195021
930,546,Oval-Shaped Genuine Coral 18k Yellow Gold Over Sterling Silver Cluster Cocktail Ring,4.194896
2197,505,PalmBeach Jewelry 5.81 TCW Oval Cut Aurora Borealis Cubic Zirconia Sterling Silver Cocktail Ring,4.194834
3830,443,Silver Chinese Ring,4.194798


In [16]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
# 该用户历史评分的产品
user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode','title', 'score']].reset_index(drop=True)
display(user_rated_products)
# 用各个模型给他推荐产品
for model, model_name in zip(models, model_names):
    recommend(sample_userid,model,model_name)

sample_userid 15996


,productId_encode,title,score
0,1111,"14k White Gold 3mm Milgrain Half Round Plain Wedding Band, Size 11",5.0
1,1106,"14k White Gold 3mm Milgrain Half Round Plain Wedding Band, Size 7.5",5.0
2,1112,"14k White Gold 3mm Milgrain Half Round Plain Wedding Band, Size 11.5",5.0
3,1108,"14k White Gold 3mm Milgrain Half Round Plain Wedding Band, Size 8.5",5.0
4,1113,"14k White Gold 3mm Milgrain Half Round Plain Wedding Band, Size 12",5.0
5,1110,"14k White Gold 3mm Milgrain Half Round Plain Wedding Band, Size 10",5.0


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_all_together_mlp
6202,15775,Invicta Men's 2235 Lupah Collection Grand Chronograph Watch,3.717895
6720,15797,Invicta Men's 2690 Pro Diver Collection Abyss Automatic Watch,3.717884
4899,15753,New Sterling Silver Filigree Floral Toering,3.717884
4292,15832,"Heavy Sterling Silver Sunflower or Flower Link Bracelet - 7""",3.717883
8196,15835,Unique Celtic Knot Maple Leaf Large Sterling Silver Earrings,3.717882
3499,15808,Invicta Men's 2844 Pro Diver Collection Automatic Watch,3.717876
5034,15891,Cute Stretchy Hematite Bracelet,3.717875
6722,15840,Belleek Shamrock Ring Holder,3.717875
3487,15860,Sterling Silver Amethyst CZ Ring with CZ Design,3.717873
7342,15916,4 Leaf Clover - Belly Ring - Luck of the Irish - Quality Materials - Customize Your Fit,3.717872


267/267 [==============================] - 0s 2ms/step


...

,productId_encode,title,predictions_user_product_rnn_mlp
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.562542
5281,36060,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 6, #10825",4.558325
7387,36062,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 7, #10825",4.557562
6021,6293,Seymour Duncan STHR1B Hot Rails Lead Pickup for Telecaster,4.554068
3448,35921,Loving Family&reg Large Sterling Silver Pendant with Extender - Parents and 3 Children,4.553396
6174,33043,"Harley-Davidson Bulova Mens Watch, Antique Medallion Skull Dial, 76A11",4.551786
5491,17245,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10, #11873",4.551455
5339,23964,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10 1/2, #11873",4.551409
3607,17248,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 11, #11873",4.551387
6813,33726,"Sterling Silver Round Clear Cubic Zirconia Initial ""M"" Charm 1 inch Long Pendant",4.550843


267/267 [==============================] - 1s 2ms/step


,productId_encode,title,predictions_model_alltogether_mlp_BERT
3200,18,Music Maker Lap Harp,4.237603
4024,55,Yamaha YRS24B Soprano Recorder,4.237278
4222,15,"Remo Kid's Percussion, Bongos, 5/6 Diameters, Rain Forest Fabric",4.236915
3783,403,PalmBeach Jewelry 18k Yellow Gold-Plated Two-Tone Filigree Butterfly Ring,4.236704
3153,283,Audio Technica ATR288W VHF Battery-Powered TwinMic Microphone System,4.236551
2196,285,Sony ECM-DS70P Electret Condenser Stereo Microphone,4.236536
7443,464,4.90 TCW Oval Cut Genuine Smoky Quartz 14k Yellow Gold Over Sterling Silver 3-Stone Ring,4.236475
6328,51,Little Tikes Tap A Tune Xylophone,4.236438
130,291,Audio-Technica ATR-35S Lavalier Microphone,4.236423
5931,6,Mechanical and Quartz Watch Repair,4.236347


267/267 [==============================] - 1s 2ms/step


,productId_encode,title,predictions_model_user_product_mlp_BERT
4883,442,Silver Chinese Ring,4.212804
8435,445,Silver Chinese Ring,4.212792
7376,446,Silver Chinese Ring,4.212789
3756,447,Silver Chinese Ring,4.212785
6097,514,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.212686
4013,517,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.212662
930,546,Oval-Shaped Genuine Coral 18k Yellow Gold Over Sterling Silver Cluster Cocktail Ring,4.212465
2197,505,PalmBeach Jewelry 5.81 TCW Oval Cut Aurora Borealis Cubic Zirconia Sterling Silver Cocktail Ring,4.212310
3830,443,Silver Chinese Ring,4.212219
2299,444,Silver Chinese Ring,4.212215


In [17]:
# 抽取其中一个用户 ID
sample_userid = np.random.choice(users_with_many_ratings)
print('sample_userid',sample_userid)
# 该用户历史评分的产品
user_rated_products = df_test[df_test['userId_encode'] == sample_userid][['productId_encode','title', 'score']].reset_index(drop=True)
display(user_rated_products)
# 用各个模型给他推荐产品
for model, model_name in zip(models, model_names):
    recommend(sample_userid,model,model_name)

sample_userid 26596


,productId_encode,title,score
0,9148,"Clarke Pennywhistle, C",4.0
1,8548,Johnson Mandolin A-Style with F Holes Sunburst,5.0
2,9154,Gibson Practice Chanter Reed,5.0
3,5986,D'Andrea Locking Nylon Straps Black,1.0
4,9152,Gibson Long Practice Chanter,5.0
5,21977,"Rosewood Practice Chanter, Book & CD",2.0


267/267 [==============================] - 0s 2ms/step


,productId_encode,title,predictions_all_together_mlp
370,25118,Excel 2'' Nylon Guitar Strap Black,3.718444
6830,25092,Le Chateau Women's 13205L_BLK Textured Dial Leather Band Watch,3.718441
3879,25055,Citizen Women's EW0732-59D Eco-Drive Corso Diamond Watch,3.718439
4850,25109,Behringer ADA8000 Ultragain Pro 8-Channel A/D & D/A Converter with Premium Mic Preamplifiers,3.718435
1726,25060,Citizen Women's EG2450-53E Eco-Drive Silhouette Bangle Watch,3.718416
5302,25047,Citizen Men's AR1062-03P Eco-Drive Stiletto Gold-Tone Watch,3.718412
3573,25066,Citizen Women's EP5614-51D Eco-Drive Lucca Diamond Accented Watch,3.718409
7222,25044,Citizen Men's BW0170-59A Eco-Drive Stainless Steel Watch,3.718407
4908,25172,Mighty Might MM1306C Tremolo Arm For Floyd Rose II with Bushing - Chrome,3.718373
1869,25031,Citizen Men's AR1060-09A Eco-Drive Stiletto Leather Watch,3.718355


267/267 [==============================] - 0s 2ms/step


...

,productId_encode,title,predictions_user_product_rnn_mlp
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.563175
5281,36060,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 6, #10825",4.558850
7387,36062,"Southwestern Style Horse & Pony Story Band Ring in Sterling Silver for Men or Women, size 7, #10825",4.558084
6021,6293,Seymour Duncan STHR1B Hot Rails Lead Pickup for Telecaster,4.554513
3448,35921,Loving Family&reg Large Sterling Silver Pendant with Extender - Parents and 3 Children,4.553916
6174,33043,"Harley-Davidson Bulova Mens Watch, Antique Medallion Skull Dial, 76A11",4.552290
5491,17245,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10, #11873",4.551903
3607,17248,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 11, #11873",4.551834
5339,23964,"Southwestern Style Animal Track ( Raccoon, Moose, Mountain Lion, Coyote, Beaver, Gray Wolf and Bear ) Band Ring in Sterling Silver with Turquoise Chip Inlay for Men or Women, size 10 1/2, #11873",4.551830
6813,33726,"Sterling Silver Round Clear Cubic Zirconia Initial ""M"" Charm 1 inch Long Pendant",4.551344


267/267 [==============================] - 1s 2ms/step


,productId_encode,title,predictions_model_alltogether_mlp_BERT
4222,15,"Remo Kid's Percussion, Bongos, 5/6 Diameters, Rain Forest Fabric",4.232956
7453,16,Harmonica,4.232913
5931,6,Mechanical and Quartz Watch Repair,4.232896
3767,11,CITIZEN Men's Silver/Gold-Tone Formal/Fashion 'STILETTO ECO-DRIVE' Watch Model AR1034-50E,4.232887
1140,12,Citizen Men's Eco-Drive Stiletto watch #AR1004-51E,4.232813
3698,19,Hohner Kids Beginner Harmonica,4.232777
5445,92,Suunto Yachtsman Wrist Top Computer Watch with Barometer and Compass,4.232657
6328,51,Little Tikes Tap A Tune Xylophone,4.232631
7465,178,"Fender 18 Ft. Vintage Voltage Guitar Cable, Black/Silver",4.232626
3706,56,Yamaha C40 Full Size Nylon-String Classical Guitar,4.232624


267/267 [==============================] - 1s 2ms/step


,productId_encode,title,predictions_model_user_product_mlp_BERT
4883,442,Silver Chinese Ring,4.200040
8435,445,Silver Chinese Ring,4.200021
7376,446,Silver Chinese Ring,4.200016
3756,447,Silver Chinese Ring,4.200009
6097,514,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.199837
4013,517,PalmBeach Jewelry Marquise-Shaped Genuine Tiger&apos;s-Eye 14k Yellow Gold-Plated Cocktail Ring,4.199805
930,546,Oval-Shaped Genuine Coral 18k Yellow Gold Over Sterling Silver Cluster Cocktail Ring,4.199483
2197,505,PalmBeach Jewelry 5.81 TCW Oval Cut Aurora Borealis Cubic Zirconia Sterling Silver Cocktail Ring,4.199254
3830,443,Silver Chinese Ring,4.199118
2299,444,Silver Chinese Ring,4.199112
